In [3]:
import torch
import torch.optim as optim
import numpy as np

torch.manual_seed(0)


In [5]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")
sentence

"if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea."

In [8]:
char_set = list(set(sentence))
char_dic = {c: i for i, c in enumerate(char_set)}

print(char_set)
print(char_dic)

['l', 'o', 'y', 'm', 'i', 's', 't', 'n', "'", 'r', '.', 'h', ' ', 'f', 'k', 'b', 'u', 'g', 'c', 'd', 'e', 'a', 'p', ',', 'w']
{'l': 0, 'o': 1, 'y': 2, 'm': 3, 'i': 4, 's': 5, 't': 6, 'n': 7, "'": 8, 'r': 9, '.': 10, 'h': 11, ' ': 12, 'f': 13, 'k': 14, 'b': 15, 'u': 16, 'g': 17, 'c': 18, 'd': 19, 'e': 20, 'a': 21, 'p': 22, ',': 23, 'w': 24}


In [9]:
# hyper parameters
dic_size = len(char_dic) #25개
hidden_size = len(char_dic)#25개
sequence_length = 10  # Any arbitrary number (우리가 지정)
learning_rate = 0.1

In [10]:
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)
#x_data와 y_data를 index형태로 바꿔준다.
    x_data.append([char_dic[c] for c in x_str])  # x str to index
    y_data.append([char_dic[c] for c in y_str])  # y str to index
#x를 one_hot vector로 만들어준다.
x_one_hot = [np.eye(dic_size)[x] for x in x_data]

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [13]:
# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [14]:
# declare RNN + FC
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

# 25개의 input이 들어가고, 25개의 hidden 사이즈?
net = Net(dic_size, hidden_size, 2)


In [15]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [41]:
# start training
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X) # 170 * 10 * 25
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2) # 170 * 10
    predict_str = ""
    for j, result in enumerate(results):
        # print(i, j, ''.join([char_set[t] for t in result]), loss.item())
        if j == 0:
            predict_str += ''.join([char_set[t] for t in result])
        else:
            predict_str += char_set[result[-1]]

    print(predict_str)

  t    a   d   ui'd tnthi tnto 't toet to  entle the   e  to t  lo   ao  tand do  t d diin toe  to  iitnd do  thtud do heo to  tnthe  to te t tod toe tnd dn ito  e  io  n toe t   
 rt apo  d do  ui d anthipt ton' hdout do  onile the   e  to to lu k ao  han  do ithao iin the  to kiitnd do  thandha  he  to  oethe  to tonl th  the  od en ito  n  io an toe t d 
 rp apoa d t  tuild anthipt ton'  dout dm tentle the   e  to tonl  p ao  hand do  t do iin them to kiitnd do  thdndhan hem to mhethem to t n' toa themtod e  ito  n  to an toe t d 
 rpoap an  to leild anthip, tonst aoet amtte dhemthe t em to l dle t ao   an  do st amsiindthem to kiitnd tor t dndhanthem ton t toem to le l tor themt d e   tor esito an toemt d 
 rtonp ens to lutle ant dp, tonst aont amtte p   the them to lodlo t aoo  and aonst amsitnsthem to ki tnd tor t dnd a them ton t toem tonle l tod themt d ess ton esstooan toemt d 
lrtont ons toeludld ant dp, tonst dont arttonp e toe them to todlo t aoos and donst drsitnsther to k

m you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
g you want to build a ship, don't drum up people together te collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
f you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the sndless immensity of the sea.
m tou want to build a ship, don't drum up people together to collect wood and don't assign them tosks and work, but rather toach them to long for themendless immensity of the eea.
p tou want to build a ship, don't drum up people together te lollect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
p tou want to build a ship, don't drum up people to ether to collect wood and don't assign them task

m you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
m you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
